In [1]:
%run ./"0 Imports and functions.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files
Cubes will be saved and loaded with Directory ./Spectral Cubes
Plots will be saved to Directory ./Plots
Dendrograms will be saved to Directory ./Dendrograms


In [2]:
# Pointing_Information

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "CMZ_3mm_HCO+.fits" #the name of the initial SC file.
File_Descriptor="cmz_3mm_HCO+_J1-0_"
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "CMZ"#
Pointing_Information["center"] = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic') #the center of the cmz
Pointing_Information["crop_center"] = SkyCoord('00d30m30.00s  ', '-00d02m46.176s', frame='galactic') #the center of the image, used for cropping

desired_beam_size = 4.3*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 8.178*10**3*u.pc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=0*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=0*u.deg
Pointing_Information["target_velocity"]=0*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 3.4*u.km/u.s
ovs = 10 #how much do you desire to oversample the beam by
desired_FOV = [70,360]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'gc_850micron_dust.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect

done


<function gc.collect(generation=2)>

In [3]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 
    
beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec)


Pointing_Information["original_BMAJ"]=beam_major
Pointing_Information["original_BMIN"]=beam_minor
Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


#this accounts for elliptical beams:            
Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])


# 1. Reprojection

In [4]:
gc.collect()

#
# def Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg]):

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=20,Cube_Name_Save='',Force_Origin=[True,[358,2],[-.6,.6]])

# This is the cube name that they are being saved to unless you changed it inside the reproject functions definiton

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")


begin step: 0 of 17
start beam convolution
0.0 0.0
0.0007902974658322842 0.0007902974658322842
convolve end

fov crop start 

Center: <SkyCoord (Galactic): (l, b) in deg
    (0.50833333, -0.04616)> Pixel center: 391 80 Pixel FOV: [73, 378]
cropping cube. rotation: 0.0 deg center: <SkyCoord (Galactic): (l, b) in deg
    (0.50833333, -0.04616)> crop to: [ 70. 360.] pc
Cropped to  [ 70. 360.] pc
fov cropped

Delete LBOUND if there is one in the header, if not this will pass the error:
"Keyword 'LBOUND1' not found."
------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_470523/3168533132.py", line 219, in Reproject_To_Region
    del reheader['LBOUND1']
  File "/home/ben/.local/lib/python3.8/site-packages/astropy/io/fits/header.py", line 239, in __delitem__
    raise KeyError(f"Keyword '{key}' not found.")
KeyError: "Keyword 'LBOUND1' not found."

start reprojection

check max SC value: 0.21000815169411258 SC shape: (1, 144, 74

# 1a. Splice the reprojected cubes back together

In [5]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

Loaded 0_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 1_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 2_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 3_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 4_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 5_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 6_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 7_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 8_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 9_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 10_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 11_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 12_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 13_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 14_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x

# 1b. Velocity reprojection

In [6]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

Start velocity reprojection of 4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected.fits
Smoothed to Gaussian Kernel of width 1.2217525938662175
Wrote reprojected cube to Spectral Cubes/4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected_3.4_vel_res_.fits


0

# 1c. Fix repeated pixels

In [7]:
# after the reprojection, there may be duplicated data from the fact that it just fills in stuff 
# expecting new data to be there. This fixes it by checking to see if it matches other data, then removes
# the repeated slices. in other words, it may be set to reproject to 0-500 km/s through the velocity channels
# but there may only be data between 100-400, so it will fill it in with copied data channels. This will check for those.



# Update to the current cube name

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Run the function to crop the nan values out of the cube

Remove_Repeated_Pixels(Cube_Name_Save)

Good data starting from channel 0 ; start has been cropped to that channel
Good data ending at channel 145 ; end has been cropped to that channel
Cropped cube saved as  Spectral Cubes/Cropped_4.3pc_beam_cmz_3mm_HCO+_J1-0_70.0x360.0pc_reprojected_3.4_vel_res_.fits


# x. save cube information

In [8]:
# Now that all the data reduction is concluded, save information about the new cubes

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)

print(Cube_Information)

{'Original_File_Name': 'CMZ_3mm_HCO+.fits', 'File_Descriptor': 'cmz_3mm_HCO+_J1-0_', 'target': 'CMZ', 'center': <SkyCoord (Galactic): (l, b) in deg
    (359.94423333, -0.04616)>, 'crop_center': <SkyCoord (Galactic): (l, b) in deg
    (0.50833333, -0.04616)>, 'distance': <Quantity 0.008178 Mpc>, 'target_image_rotation': <Quantity 0. deg>, 'target_inclination': <Quantity 0. deg>, 'target_velocity': <Quantity 0. km / s>, 'vaxis': 0, 'desired_velocity_resolution': <Quantity 3.4 km / s>, 'Corresponding_Continuum': 'gc_850micron_dust.fits', 'wavelength': <Quantity 0.00336133 m>, 'restfreq': <Quantity 8.9188526e+10 Hz>, 'CUNIT': <Quantity 1. deg>, 'original_BMAJ': <Quantity 38.98800015 arcsec>, 'original_BMIN': <Quantity 38.98800015 arcsec>, 'original_BMAJ_pc': <Quantity 1.54579868e-06 Mpc rad>, 'original_BMIN_pc': <Quantity 1.54579868e-06 Mpc rad>, 'desired_beam_size': <Quantity 4.3 pc>, 'original_pixel_scale_x': <Quantity 12.00000074 arcsec / pix>, 'original_pixel_scale_y': <Quantity 12.000